In [2]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('device:', device)

device: cuda


In [5]:
BATCH_SIZE = 64

train_data = torchvision.datasets.CIFAR10(
	  root='./data',
	  train=True,
	  transform=torchvision.transforms.ToTensor(),
	  download=True,
)

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=torchvision.transforms.ToTensor())

train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
channel_size_1 = 16
channel_size_2 = 32

Extracting ./data/cifar-10-python.tar.gz to ./data


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=channel_size_1,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.BatchNorm2d(channel_size_1),
            nn.ReLU(True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(channel_size_1, channel_size_1, 3, 1, 1),
            nn.BatchNorm2d(channel_size_1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(channel_size_1, channel_size_2, 3, 1, 1),
            nn.BatchNorm2d(channel_size_2),
            nn.ReLU(True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(channel_size_2, channel_size_2, 3, 1, 1),
            nn.BatchNorm2d(channel_size_2),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(channel_size_2, channel_size_2, 3, 1, 1),
            nn.BatchNorm2d(channel_size_2),
            nn.ReLU(True),
        )
        self.out = nn.Sequential(
            #nn.Conv2d(channel_size_2, 10, 1, 1, 0),
            #nn.AvgPool2d(8)
            nn.Flatten(),
            nn.Linear(channel_size_2 * 8 * 8, 10)
        )
        self.conv_layers = [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5]

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
            
        output = self.out(x).view(-1, 10)
        return output

In [9]:
cnn = CNN().to(device)

cnn

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride

In [10]:
optimizer = torch.optim.Adam(cnn.parameters())
loss_func = nn.CrossEntropyLoss()

In [11]:
n_epochs = 10

for epoch in range(n_epochs):
    print('Epoch {}/{}'.format(epoch, n_epochs))

    